In [1]:
import regression
import tests.utils
import numpy as np

In [2]:
x, y, y_true, reference_model = tests.utils.data_generation_model(100, 3)

NameError: name 't' is not defined

In [ ]:
verify = reference_model.data_matrix(x)

In [ ]:
verify

In [ ]:
v = np.zeros((10, 10), dtype=int)
t = np.zeros((10, 10))
hinge = v > 1
where = v > 1

In [ ]:
for i, basis in enumerate(reference_model.basis):
    v[:len(basis.v), i] = basis.v
    t[:len(basis.t), i] = basis.t
    hinge[:len(basis.hinge), i] = basis.hinge
    where[:len(basis.hinge), i] = True

In [ ]:
result = -t + x[:, v] 
np.maximum(np.zeros_like(result), result, where=hinge, out=result)
result = result.prod(axis=1, where=where)#[:, :len(reference_model.basis)]
result